In [2]:
import pyawr.mwoffice as mwo
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation 

'***PICK ONE***'
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
# %matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
%matplotlib tk 

In [3]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #Invoke circuit simulator analysis

In [4]:
graph_name = awrde.Project.Graphs.Item(2).Name
graph = awrde.Project.Graphs(graph_name)
meas = graph.Measurements[0]

num_pts = meas.XPointCount
xs = ys = dys = np.zeros(num_pts)

In [5]:
#Let's try to interpolate:
from scipy.interpolate import make_interp_spline, BSpline

In [6]:
def animate_smooth(n_frm): #Animate method for FuncAnimation
    xs = np.asarray(meas.XValues)
    ys = np.asarray(meas.YValues(1))
    dys = np.gradient(ys, xs)
    abs_dys = np.abs(dys)

    new_xs = np.linspace(xs.min(), xs.max(), 300)

    spl_ys = make_interp_spline(xs, ys, k=7)  # type: BSpline
    spl_dys = make_interp_spline(xs, dys, k=7)
    spl_abs_dys = make_interp_spline(xs, abs_dys, k=7)

    ys_smooth = spl_ys(new_xs)
    dys_smooth = spl_dys(new_xs)
    abs_dys_smooth = spl_abs_dys(new_xs)

    axs_smooth[0, 0].cla()
    axs_smooth[1, 0].cla()
    axs_smooth[2, 0].cla()
    axs_smooth[0, 0].plot(xs, ys, 'r.')
    axs_smooth[1, 0].plot(xs, dys, 'b.')
    axs_smooth[2, 0].plot(xs, abs_dys, 'g.')

    axs_smooth[0, 1].cla()
    axs_smooth[1, 1].cla()
    axs_smooth[2, 1].cla()
    axs_smooth[0, 1].plot(new_xs, ys_smooth, 'r')
    axs_smooth[1, 1].plot(new_xs, dys_smooth, 'b')
    axs_smooth[2, 1].plot(new_xs, abs_dys_smooth, 'g')

In [7]:
# TODO: Get zoom to work?
fig_smooth, axs_smooth = plt.subplots(3, 2, sharex='all', sharey='row')


#TODO: Figure out this blit thing
ani = FuncAnimation(fig_smooth, animate_smooth, interval=200, blit=False) #Create animation object. blit=True is for smoother animations, only changed data should be updated.
plt.show()